In [1]:
# Import Dependencies
import pandas as pd
from datetime import datetime as dt
from bs4 import BeautifulSoup as bs
import requests
import os
import numpy as np
from splinter import Browser

# Google Office Parsing

In [50]:
url='https://about.google/locations/?region=true'
response = requests.get(url)

In [51]:
soup = bs(response.text, 'html.parser')

In [52]:
results = soup.find_all("div", class_= "office-info")
results

[<div class="office-info">
 <h2 class="office-name" itemprop="name">
                   Dubai
                 </h2>
 <div class="office-address" itemprop="address">TECOM Zone, Dubai Internet City
 Dubai, United Arab Emirates</div>
 <div class="office-phone-number">
                     Phone:
                     <span class="phone-number" data-phone-number="+971 4 4509500" itemprop="telephone">
                       +971 4 4509500
                     </span>
 </div>
 <div class="directions">
 <a href="https://www.google.com/maps/dir/Current+Location/25.0929,55.1591?hl=en" rel="noopener" target="_blank">
                     Directions
                   </a>
 </div>
 </div>,
 <div class="office-info">
 <h2 class="office-name" itemprop="name">
                   Haifa
                 </h2>
 <div class="office-address" itemprop="address">Building 30
 MATAM, Advanced Technology Center
 Haifa, 3190500
 Israel </div>
 <div class="office-phone-number">
                     Phone:
      

In [53]:
results = soup.find_all("div", class_= "office-info")
for result in results:
    office = result.find("h2", itemprop= "name").text
    address = result.find("div",itemprop= "address").text
    quards = result.find("a")['href']
    print("-------------------")
    print(office, address, quards)
    
    

-------------------

                  Dubai
                 TECOM Zone, Dubai Internet City
Dubai, United Arab Emirates https://www.google.com/maps/dir/Current+Location/25.0929,55.1591?hl=en
-------------------

                  Haifa
                 Building 30
MATAM, Advanced Technology Center
Haifa, 3190500
Israel  https://www.google.com/maps/dir/Current+Location/32.78897,34.958432?hl=en
-------------------

                  Istanbul
                 Eski Buyukdere Caddesi No: 209
34394
Istanbul, Turkey https://www.google.com/maps/dir/Current+Location/41.081697,29.00859?hl=en
-------------------

                  Johannesburg
                 35 Ballyclare Drive Building E
Johannesburg
2191, South Africa https://www.google.com/maps/dir/Current+Location/-26.0734457,28.032035?hl=en
-------------------

                  Tel Aviv
                 Yigal Alon 98
Tel Aviv, 6789141
Israel  https://www.google.com/maps/dir/Current+Location/32.070043,34.794087?hl=en
-------------------


In [54]:
location = []
addresses = []
quardss = []

for result in results:
    office = result.find("h2", class_= "office-name").text
    address = result.find("div", class_= "office-address").text
    quards = result.find("a")['href']
    location.append(office)
    addresses.append(address)
    quardss.append(quards)
    

In [55]:
# Create a dataframe from the parsed data above

df = pd.DataFrame({
    "Office": location,
    "Address": addresses,
    "Quards": quardss
})

# Removes all occurances of "\n"
df["Office"].replace(r'\s+|\\n', ' ', regex=True, inplace=True) 
df["Address"].replace(r'\s+|\\n', ' ', regex=True, inplace=True) 

#Drops the:'https://www.google.com/maps/dir/Current+Location/ at the beggining of the string
# and the: ?hl=en at the end of the string
df["Quards2"]= df["Quards"].str.strip('https://www.google.com/maps/dir/Current+Location/ ?hl=en')

# Drops the old Column with the google hyperlinks
df = df.drop(columns=['Quards'])


# new data frame with split value columns 
new = df["Quards2"].str.split(",", n = 1, expand = True) 
  
# making separate Latitude column from new data frame 
df["Latitude"]= new[0] 
  
# making separate Longitude column from new data frame 
df["Longitude"]= new[1] 
  
# Dropping old Quards2 columns 
df.drop(columns =["Quards2"], inplace = True) 
  
# Drops duplicates and only keeps the Office's first occurance
df= df.drop_duplicates(subset=["Office"], keep='first')

In [56]:
df["Company"] = ("Google")

In [57]:
df.set_index("Company")

,Office,Address,Latitude,Longitude
Company,,,,
Google,Dubai,"TECOM Zone, Dubai Internet City Dubai, United ...",25.0929,55.1591
Google,Haifa,"Building 30 MATAM, Advanced Technology Center ...",32.78897,34.958432
Google,Istanbul,"Eski Buyukdere Caddesi No: 209 34394 Istanbul,...",41.081697,29.00859
Google,Johannesburg,35 Ballyclare Drive Building E Johannesburg 21...,-26.0734457,28.032035
Google,Tel Aviv,"Yigal Alon 98 Tel Aviv, 6789141 Israel",32.070043,34.794087
...,...,...,...,...
Google,San Francisco,"345 Spear Street San Francisco, CA 94105 Unite...",37.789972,-122.390013
Google,Seattle,"601 N. 34th Street Seattle, WA 98103 United St...",47.649316,-122.350629
Google,Sunnyvale,"803 11th Avenue Sunnyvale, CA 94089 United States",37.403694,-122.031583


# Amazon Office Parsing

In [2]:
executable_path = {'executable_path': "chromedriver.exe"}
browser = Browser('chrome', **executable_path, headless=False)

In [68]:
amazonurl='https://craft.co/amazon/locations?page='
pg_num = ['2', '3', '4', '5', '6', '7', '8']
browser.visit(amazonurl)

In [63]:
html = browser.html
print(html)

<html class=""><head><link href="//assets.production.craft.co/static/img/favicons/production/apple-touch-icon.png" rel="apple-touch-icon" sizes="60x60"><link href="//assets.production.craft.co/static/img/favicons/production/apple-touch-icon.png" rel="apple-touch-icon" sizes="76x76"><link href="//assets.production.craft.co/static/img/favicons/production/apple-touch-icon.png" rel="apple-touch-icon" sizes="120x120"><link href="//assets.production.craft.co/static/img/favicons/production/apple-touch-icon.png" rel="apple-touch-icon" sizes="152x152"><link href="//assets.production.craft.co/static/img/favicons/production/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"><link href="//assets.production.craft.co/static/img/favicons/production/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"><link href="//assets.production.craft.co/static/img/favicons/production/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"><link href="//assets.production.craft.co/static/img/

In [70]:
for num in pg_num:
    queryamazonurl = browser.visit(amazonurl + '2')
    browser.visit(amazonurl)
    amazonresults = amazonsoup.find_all("div", class_= "location__city_and_state")
    for amazonresult in amazonresults:
            office = amazonresult.find("span", itemprop= "addressLocality").text
            address = amazonresult.find("span",itemprop= "streetAddress")
            country = amazonresult.find("div", class_="location_country")
            print("-------------------")
            print(num, office, address, country)  


-------------------
2 Seattle None None
-------------------
2 Arlington None None
-------------------
2 Atlanta None None
-------------------
2 Austin None None
-------------------
2 Baltimore None None
-------------------
2 Bellevue None None
-------------------
2 Boardman None None
-------------------
2 Boulder None None
-------------------
2 Breinigsville None None
-------------------
2 Brooklyn None None
-------------------
2 Caledonia None None
-------------------
2 Campbellsville None None
-------------------
2 Campbellsville None None
-------------------
2 Carlisle None None
-------------------
2 Carteret None None
-------------------
2 Charleston None None
-------------------
2 Charlotte None None
-------------------
2 Chattanooga None None
-------------------
2 Chester None None
-------------------
2 Chicago None None
-------------------
2 Columbus None None
-------------------
2 Coppell None None
-------------------
2 Cupertino None None
-------------------
2 Dallas None None

In [65]:
amazonsoup = bs(html, "html.parser")
print(amazonsoup.prettify())

<html class="">
 <head>
  <link href="//assets.production.craft.co/static/img/favicons/production/apple-touch-icon.png" rel="apple-touch-icon" sizes="60x60"/>
  <link href="//assets.production.craft.co/static/img/favicons/production/apple-touch-icon.png" rel="apple-touch-icon" sizes="76x76"/>
  <link href="//assets.production.craft.co/static/img/favicons/production/apple-touch-icon.png" rel="apple-touch-icon" sizes="120x120"/>
  <link href="//assets.production.craft.co/static/img/favicons/production/apple-touch-icon.png" rel="apple-touch-icon" sizes="152x152"/>
  <link href="//assets.production.craft.co/static/img/favicons/production/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
  <link href="//assets.production.craft.co/static/img/favicons/production/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
  <link href="//assets.production.craft.co/static/img/favicons/production/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
  <link href="//asset

In [71]:
for num in pg_num:
    queryamazonurl = amazonurl + '2'
    browser.visit(amazonurl)
    amazonresults = amazonsoup.find_all('div', itemprop="location")
    for amazonresult in amazonresults:
        office = amazonresult.find(itemprop= "addressLocality").text
        address = amazonresult.find(itemprop="streetAddress").text
        state = amazonresult.find(class_="location__state").text
        country = amazonresult.find(class_="location__country").text
        print("-------------------")
        print(num, office, address, state, country)   

-------------------
2 Seattle 410 Terry Ave N WA United States
-------------------
2 Arlington Crystal City VA United States
-------------------
2 Atlanta 3333 Piedmont Rd NE GA United States
-------------------
2 Austin 11501 Alterra Pkwy TX United States
-------------------
2 Baltimore 2010 Broening Hwy MD United States
-------------------
2 Bellevue 425 106th Ave NE WA United States
-------------------
2 Boardman 73577 Lewis and Clark Dr OR United States
-------------------
2 Boulder 0009279 CO United States
-------------------
2 Breinigsville 705 Boulder Dr PA United States
-------------------
2 Brooklyn 850 3rd Ave, Brooklyn NY United States
-------------------
2 Caledonia 4500 68th St SE MI United States
-------------------
2 Campbellsville 1051 S Columbia Ave KY United States
-------------------
2 Campbellsville 1188 S Columbia Ave KY United States
-------------------
2 Carlisle 40 Logistics Dr PA United States
-------------------
2 Carteret 308 Middlesex Ave NJ United States
--

-------------------
6 Seattle 410 Terry Ave N WA United States
-------------------
6 Arlington Crystal City VA United States
-------------------
6 Atlanta 3333 Piedmont Rd NE GA United States
-------------------
6 Austin 11501 Alterra Pkwy TX United States
-------------------
6 Baltimore 2010 Broening Hwy MD United States
-------------------
6 Bellevue 425 106th Ave NE WA United States
-------------------
6 Boardman 73577 Lewis and Clark Dr OR United States
-------------------
6 Boulder 0009279 CO United States
-------------------
6 Breinigsville 705 Boulder Dr PA United States
-------------------
6 Brooklyn 850 3rd Ave, Brooklyn NY United States
-------------------
6 Caledonia 4500 68th St SE MI United States
-------------------
6 Campbellsville 1051 S Columbia Ave KY United States
-------------------
6 Campbellsville 1188 S Columbia Ave KY United States
-------------------
6 Carlisle 40 Logistics Dr PA United States
-------------------
6 Carteret 308 Middlesex Ave NJ United States
--

In [26]:
amazonresults = amazonsoup.find_all('div', class_="location")
print(amazonresults)




[<div class="location"><div class="location__city_and_state"><span itemprop="addressLocality">Seattle</span><span class="location__hq"> (HQ)</span>, <span class="location__state">WA</span></div><div class="location__country" itemprop="addressCountry">United States</div><address class="location__address location__address--fadeout"><span itemprop="streetAddress">410 Terry Ave N</span> <span> </span></address></div>, <div class="location"><div class="location__city_and_state"><span itemprop="addressLocality">Arlington</span>, <span class="location__state">VA</span></div><div class="location__country" itemprop="addressCountry">United States</div><address class="location__address location__address--fadeout"><span itemprop="streetAddress">Crystal City</span> <span> </span></address></div>, <div class="location"><div class="location__city_and_state"><span itemprop="addressLocality">Atlanta</span>, <span class="location__state">GA</span></div><div class="location__country" itemprop="addressCou

NameError: name 'span' is not defined

In [34]:
for div in amazonsoup.find_all("div", class_= "location__city_and_state"):
    for span in div.find_all("spalocation__city_and_staten", itemprop= "streetAdress"):
        print(span)

In [36]:
for div in amazonsoup.find_all("span", itemprop= "streetAddress"):
    print(div)

<span itemprop="streetAddress">410 Terry Ave N</span>
<span itemprop="streetAddress">Crystal City</span>
<span itemprop="streetAddress">3333 Piedmont Rd NE</span>
<span itemprop="streetAddress">11501 Alterra Pkwy</span>
<span itemprop="streetAddress">2010 Broening Hwy</span>
<span itemprop="streetAddress">425 106th Ave NE</span>
<span itemprop="streetAddress">73577 Lewis and Clark Dr</span>
<span itemprop="streetAddress">0009279</span>
<span itemprop="streetAddress">705 Boulder Dr</span>
<span itemprop="streetAddress">850 3rd Ave, Brooklyn</span>
<span itemprop="streetAddress">4500 68th St SE</span>
<span itemprop="streetAddress">1051 S Columbia Ave</span>
<span itemprop="streetAddress">1188 S Columbia Ave</span>
<span itemprop="streetAddress">40 Logistics Dr</span>
<span itemprop="streetAddress">308 Middlesex Ave</span>
<span itemprop="streetAddress">225 Infinity Dr</span>
<span itemprop="streetAddress">300 Dalton Ave</span>
<span itemprop="streetAddress">7200 Discovery Drive</span>
<